# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv(r'C:\Users\paris\Desktop\DSI_demoC4\production\05_src\data\adult\adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
print(adult_dt)


       age          workclass  fnlwgt    education  education-num   
0       39          State-gov   77516    Bachelors             13  \
1       50   Self-emp-not-inc   83311    Bachelors             13   
2       38            Private  215646      HS-grad              9   
3       53            Private  234721         11th              7   
4       28            Private  338409    Bachelors             13   
...    ...                ...     ...          ...            ...   
32556   27            Private  257302   Assoc-acdm             12   
32557   40            Private  154374      HS-grad              9   
32558   58            Private  151910      HS-grad              9   
32559   22            Private  201490      HS-grad              9   
32560   52       Self-emp-inc  287927      HS-grad              9   

            marital-status          occupation    relationship    race   
0            Never-married        Adm-clerical   Not-in-family   White  \
1       Married-civ-spo

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

pd_adult = pd.DataFrame(adult_dt)

# Create the feature dataframe (X) and target series (Y)
X = pd_adult.drop(columns=['income'])
Y = pd_adult['income']

# Split the data into training and testing sets with a 70-30 split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Output the resulting data splits for verification
print(X_train.shape,X_test.shape)


(22792, 14) (9769, 14)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random state is a parameter used to control the randomness of the data splitting process. It essentially acts as a "seed" for the random number generator, ensuring that the data is split the same way every time the code is run.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score
from sklearn.compose import ColumnTransformer

In [18]:
Numerical_Transformer = Pipeline(
    [
        ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
        ('scaler', RobustScaler())

    ]
)
Numerical_Transformer

Pipeline(steps=[('imputer', KNNImputer(n_neighbors=7, weights='distance')),
                ('scaler', RobustScaler())])

In [19]:
Categorical_Transformer = Pipeline(
    [
        ('simpleimputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
       
    ]
)
Categorical_Transformer

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [21]:
pd_adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  int32 
dtypes: int32(1), int64(6), object(8)
memory usage: 3.6+ MB


In [48]:

transformer = ColumnTransformer(
    transformers=[
        ('num_transforms', Numerical_Transformer, ['age', 'fnlwgt','education-num','capital-gain',
                                        'capital-loss','hours-per-week'] ),
        ('cat_transforms', Categorical_Transformer, ['workclass', 'education','marital-status','occupation',
                                        'relationship','race','sex','native-country']) 
    ], 
    remainder='drop'
)

pipe = Pipeline(
    [
        ('preprocessing', transformer)
    ]
)
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])]))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Create the model pipeline
model_pipeline = Pipeline([
    ('preprocessing', transformer),       # Add the ColumnTransformer as a preprocessing step
    ('classifier', RandomForestClassifier())  # Add the RandomForestClassifier as the classifier step
])
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [50]:

from sklearn.model_selection import cross_validate

# Specify the scoring metrics
scoring = ['accuracy', 'roc_auc', 'neg_log_loss', 'balanced_accuracy']

# Run cross-validation
result = cross_validate(model_pipeline, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)

# Display results
print(result)


{'fit_time': array([14.25699234, 13.48551798, 13.15471673, 13.42700601, 13.76354074]), 'score_time': array([0.28009033, 0.29042053, 0.2525301 , 0.29554653, 0.26155567]), 'test_accuracy': array([0.85194122, 0.84865102, 0.8532251 , 0.85849057, 0.85761299]), 'train_accuracy': array([0.99994515, 0.99994515, 1.        , 0.99989031, 0.99994516]), 'test_roc_auc': array([0.90452794, 0.90282358, 0.90130754, 0.90774774, 0.90185503]), 'train_roc_auc': array([1., 1., 1., 1., 1.]), 'test_neg_log_loss': array([-0.36407151, -0.36720254, -0.3961767 , -0.35541503, -0.38183433]), 'train_neg_log_loss': array([-0.08163146, -0.08144583, -0.08143855, -0.08167742, -0.08140436]), 'test_balanced_accuracy': array([0.77504584, 0.76980103, 0.77666833, 0.77983588, 0.77771824]), 'train_balanced_accuracy': array([0.9998869 , 0.9998869 , 1.        , 0.99977386, 0.99988693])}


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [52]:
fold_level_results = pd.DataFrame(result).sort_values(by = 'test_neg_log_loss', ascending = False)
fold_level_results

,fit_time,score_time,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc,test_neg_log_loss,train_neg_log_loss,test_balanced_accuracy,train_balanced_accuracy
3,13.427006,0.295547,0.858491,0.999890,0.907748,1.0,-0.355415,-0.081677,0.779836,0.999774
0,14.256992,0.280090,0.851941,0.999945,0.904528,1.0,-0.364072,-0.081631,0.775046,0.999887
1,13.485518,0.290421,0.848651,0.999945,0.902824,1.0,-0.367203,-0.081446,0.769801,0.999887
4,13.763541,0.261556,0.857613,0.999945,0.901855,1.0,-0.381834,-0.081404,0.777718,0.999887
2,13.154717,0.252530,0.853225,1.000000,0.901308,1.0,-0.396177,-0.081439,0.776668,1.000000


Calculate the mean of each metric. 

In [53]:
fold_level_results.mean()

fit_time                   13.617555
score_time                  0.276029
test_accuracy               0.853984
train_accuracy              0.999945
test_roc_auc                0.903652
train_roc_auc               1.000000
test_neg_log_loss          -0.372940
train_neg_log_loss         -0.081520
test_balanced_accuracy      0.775814
train_balanced_accuracy     0.999887
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [56]:
# Fit the model pipeline on the training data
model_pipeline.fit(X_train, Y_train)

#Define test data
pred_test = model_pipeline.predict(X_test)
proba_test = model_pipeline.predict_proba(X_test)

from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

performance_metrics = {
    'log_loss_test': log_loss(Y_test, proba_test),
    'roc_auc_score_test': roc_auc_score(Y_test, proba_test[:, 1]),
    'accuracy_score_test': accuracy_score(Y_test, pred_test),
    'balanced_accuracy_score_test': balanced_accuracy_score(Y_test, pred_test)
}

print(performance_metrics)

{'log_loss_test': 0.3811738894640912, 'roc_auc_score_test': 0.899955828314746, 'accuracy_score_test': 0.8543351417750026, 'balanced_accuracy_score_test': 0.7740305851241126}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recoding the target variable into a binary numerical format ensures compatibility with binary classification models and simplifies evaluation.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.